Aims:
1. Finalizing data processing/cleaning function.
2. Explortatory Data Snalysis (EDA)

Before heading into EDA, data processing function needs 
to be modified for negation, url link structure variability,
and Twitter ID variability.

Negation words are split into two parts. The 't' after the apostrophe vanishes 
when I filtethe r tokens with length >1 syllable. This means "can't" = "can".

Some of the URL links don't start with "http" as initially assumed. Some people paste 
"www.aaa.com" as a link. Also, another fix is to add symbols such as "=", "_", "~", 
etc. to URL editor.

Lastly, Twitter ID's are modified further to include underscore symbol as character
that can be used with ID. 

Updated cleaning function in the following order:
    
1. Souping
2. BOM removing
3. url address('http:'pattern), twitter ID removing
4. url address('www.'pattern) removing
5. lower-case
6. negation handling
7. removing numbers and special char's
8. tokenizing and joining

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # Two lines above, during the letters_only process, unnecessay white spaces were created.
    # To remove these white space, tokenize and join together.
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [9]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv",header=None,usecols=[0,5],names=['sentiment','text'])
df['sentiment'] = df['sentiment'].map({0:0, 4:1})
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [10]:
%%time
print "Cleaning the tweets...\n"
clean_tweets_texts = []
for i in xrange(0, len(df)):
    if( (i+1)%100000 == 0 ):
        print "Tweets %d of %d has been processed" % (i+1, len(df) )
    clean_tweets_texts.append(tweet_cleaner_updated(df['text'][i]))

Cleaning the tweets...



/Users/mindyng/anaconda/lib/python2.7/encodings/utf_8_sig.py:19: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if input[:3] == codecs.BOM_UTF8:


Tweets 100000 of 1600000 has been processed
Tweets 200000 of 1600000 has been processed
Tweets 300000 of 1600000 has been processed
Tweets 400000 of 1600000 has been processed
Tweets 500000 of 1600000 has been processed
Tweets 600000 of 1600000 has been processed
Tweets 700000 of 1600000 has been processed
Tweets 800000 of 1600000 has been processed
Tweets 900000 of 1600000 has been processed
Tweets 1000000 of 1600000 has been processed
Tweets 1100000 of 1600000 has been processed
Tweets 1200000 of 1600000 has been processed
Tweets 1300000 of 1600000 has been processed
Tweets 1400000 of 1600000 has been processed
Tweets 1500000 of 1600000 has been processed
Tweets 1600000 of 1600000 has been processed
CPU times: user 11min 27s, sys: 34.1 s, total: 12min 1s
Wall time: 12min 3s
